## 目录
- [9. 信息抽取](#9-信息抽取)
- [9.1 新词提取](#91-新词提取)
- [9.2 关键词提取](#92-关键词提取)
- [9.3 短语提取](#93-短语提取)
- [9.4 关键句提取](#94-关键句提取)
- [9.5 总结](#95-总结)

## 9. 信息抽取

信息抽取是一个宽泛的概念，指的是从非结构化文本中提取结构化信息的一类技术。这类技术依然分为基于规则的正则匹配、有监督学习和无监督学习等各种实现方法。我们将使用一些简单实用的无监督学习方法。由于不需要标注语料库，所以可以利用海量的非结构化文本。

本章按照颗粒度从小到大的顺序，介绍抽取新词、关键词、关键短语和关键句的无监督学习方法。



### 9.1 新词提取

1. **概述**

   新词是一个相对的概念，每个人的标准都不一样，所以我们这里定义: 词典之外的词语(OOV)称作**新词**。

   新词的提取对中文分词而言具有重要的意义，因为语料库的标注成本很高。那么如何修订领域词典呢，此时，无监督的新词提取算法就体现了现实意义。

   

2. **基本原理**

   - 提取出大量文本(生语料)中的词语，无论新旧。
   - 用词典过滤掉已有的词语，于是得到新词。

   步骤 2 很容易，关键是步骤 1，如何无监督的提取出文本中的单词。给定一段文本，随机取一个片段，如果这个片段左右的搭配很丰富，并且片段内部成分搭配很固定，则可以认为这是一个词。将这样的片段筛选出来，按照频次由高到低排序，排在前面的有很高概率是词。

   如果文本足够大，再用通用的词典过滤掉“旧词”，就可以得到“新词”。

   片段外部左右搭配的丰富程度，可以用**信息熵**来衡量，而片段内部搭配的固定程度可以用子序列的**互信息**来衡量。

   

3. **信息熵**

   在信息论中，**信息熵**( entropy )指的是某条消息所含的信息量。它反映的是听说某个消息之后，关于该事件的不确定性的减少量。比如抛硬币之前，我们不知道“硬币正反”这个事件的结果。但是一旦有人告诉我们“硬币是正面”这条消息，我们对该次抛硬币事件的不确定性立即降为零，这种不确定性的减小量就是信息熵。公式如下:

   $$H(x)=-\sum_xP(x)logP(x)$$

   给定字符串 S 作为词语备选，X 定义为该字符串左边可能出现的字符(左邻字)，则称 H(X) 为 S 的左信息熵，类似的，定义右信息熵 H(Y)，例如下列句子:

   > 两只**蝴蝶**飞啊飞
   >
   > 这些**蝴蝶**飞走了

   那么对于字符串蝴蝶，它的左信息熵为1，而右信息熵为0。因为生语料库中蝴蝶的右邻字一定是飞。假如我们再收集一些句子，比如“蝴蝶效应”“蝴蝶蜕变”之类，就会观察到右信息熵会增大不少。

   左右信息熵越大，说明字符串可能的搭配就越丰富，该字符串就是一个词的可能性就越大。

   光考虑左右信息熵是不够的，比如“吃了一顿”“看了一遍”“睡了一晚”“去了一趟”中的了一的左右搭配也很丰富。为了更好的效果，我们还必须考虑词语内部片段的凝聚程度，这种凝聚程度由互信息衡量。

   

4. **互信息**

   **互信息**指的是两个离散型随机变量 X 与 Y 相关程度的度量，定义如下:

   $$I(X;Y)=\sum_{x,y}P(x,y)log\frac{P(x,y)}{P(x)P(y)}=E_{P(x,y)}log\frac{P(x,y)}{P(x)P(y)}$$

   互信息的定义可以用韦恩图直观表达:

   <img src="imgs/weien.png" width="200" />

   

   其中，左圆圈表示H(X)，右圆圈表示H(Y)。它们的并集是联合分布的信息熵H(X,Y)，差集有多件嫡，交集就是互信息。可见互信息越大，两个随机变量的关联就越密切，或者说同时发生的可能性越大。

   片段可能有多种组合方式，计算上可以选取所有组合方式中互信息最小的那一种为代表。有了左右信息熵和互信息之后，将两个指标低于一定阈值的片段过滤掉，剩下的片段按频次降序排序，截取最高频次的 N 个片段即完成了词语提取流程。

   

5. **实现**

   我们用四大名著来提起100个高频词。

   代码请见(**语料库自动下载**): extract_word.py

   [https://github.com/NLP-LOVE/Introduction-NLP/tree/master/code/ch09/extract_word.py](https://github.com/NLP-LOVE/Introduction-NLP/tree/master/code/ch09/extract_word.py)

   运行结果如下:

   <img src="imgs/result.png" />

   

   虽然我们没有在古典文学语料库上进行训练，但新词识别模块成功的识别出了麝月、高太尉等生僻词语，该模块也适用于微博等社交媒体的不规范文本。
   
<font color="red">

6. **实现流程【阅读源码】**

> 1. 读取整个文档，记录整个文档的总字符数m，利用标点符号进行句子分割
>
> 假设文档内容为:
>
> """两只蝴蝶飞啊飞。
>
> 这些蝴蝶飞走了。
>
> 蝴蝶效应。"""
>
> 2. 每次读取一个句子
>
> 3. 设定词的最大长度n，循环该句子，从该句子的第一个字开始记录，到第n个字，然后从第2个字开始记录，再往后到第n个字，记录下词和该词的前一个和后一个字符以及它们出现的频次。
>
>例如：“蝴蝶”的前一个“只”出现次数为1，“些”出现次数为1，还有一个前面没字符的我们统一用“\0”标识【如果最后没有字符也用这个标识】
>
><img src="imgs/treemap.png" width="400"/>
>    
> 4. 计算一个字（或者一个小于等于最大长度的词）的左右熵
>
>例如：对于“蝴蝶“的左右信息熵分别为：
>
>$$P("只")=\frac{"只"出现的次数}{整个文档的总字符数m}$$
>    
>左信息熵：$$H_{left}=(-\frac{1}{m}log\frac{1}{m})+(-\frac{1}{m}log\frac{1}{m})+(-\frac{1}{m}log\frac{1}{m})$$
>
>右信息熵：$$H_{right}=(-\frac{2}{m}log\frac{2}{m})+(-\frac{1}{m}log\frac{1}{m})$$
>
>我们取左右信息熵较小的作为熵衡量即可，熵越大，说明这个词的前后搭配类型越丰富，说明这个词是个”真正词“的可能性越大。
> 
> 5. 计算互信息，取互信息最小的一组即可，互信息越大说明字符的相关性越强，说明这个词是个”真正词“的可能性越大。
>
>例如：对于“蝴蝶“的互信息，这里正好是两个字符，所以就这么一组互信息
>
>$$P(蝴，蝶)=\frac{”蝴蝶”出现的频次}{所有词的总频次}$$
>因为我们不知道这些句子本身正确的分词方式，所以所有词的总频次我们是得不到的；这里用m来替代所有词的总频次，本身大家都一样，所以所有词的总频次取任何值都不影响结果。
> $$I(蝴；蝶)=P(蝴，蝶)log\frac{P(蝴，蝶)}{P(蝴)P(蝶)}=\frac{3}{m}log\frac{\frac{3}{m}}{\frac{3}{m}\frac{3}{m}}$$
>
>例如：三个字的词“杨柳树”
>
>$$I(杨;柳树)=P(杨，柳树)log\frac{P(杨，柳树)}{P(杨)P(柳树)}$$
>
>$$I(杨柳;树)=P(杨柳，树)log\frac{P(杨柳，树)}{P(杨柳)P(树)}$$
>
>取$I(杨;柳树)$和$I(杨柳;树)$互信息最小的一组即可，互信息越大说明字符的相关性越强，说明这个词是个”真正词“的可能性越大。计算的时候由于$P(杨，柳树)=P(杨柳，树)$，且都是取log，所以只计算$\frac{P(杨，柳树)}{P(杨)P(柳树)}\ \ \ \ $和$\frac{P(杨柳，树)}{P(杨柳)P(树)}\ \ \ \ $进行对比即可。

注意：对于互信息的计算只有X和Y是序列的时候才会计算期望，例如X=[1,2,3,4]，Y=[0,1,1,0]，此时才会有期望，这里就类似于X=[3]，Y=[0]所以没有期望，或者说期望就是它本身。
</font>

### 9.2 关键词提取

词语颗粒度的信息抽取还存在另一个需求，即提取文章中重要的单词，称为**关键词提起**。关键词也是一个没有定量的标准，无法统一语料库，所以就可以利用无监督学习来完成。

分别介绍词频、TF-IDF和TextRank算法，单文档提起可以用词频和TextRank，多文档可以使用TF-IDF来提取关键词。

1. **词频统计**

   关键词通常在文章中反复出现，为了解释关键词，作者通常会反复提及它们。通过统计文章中每种词语的词频并排序，可以初步获取部分关键词。

   不过文章中反复出现的词语却不一定是关键词，例如“的”。所以在统计词频之前需要去掉停用词。

   词频统计的流程一般是分词、停用词过滤、按词频取前 n 个。其中，求 m 个元素中前 n (n<=m) 大元素的问题通常通过最大堆解决，复杂度为 O(mlogn)。HanLP代码如下:

   ```python
   from pyhanlp import *
   
   TermFrequency = JClass('com.hankcs.hanlp.corpus.occurrence.TermFrequency')
   TermFrequencyCounter = JClass('com.hankcs.hanlp.mining.word.TermFrequencyCounter')
   
   if __name__ == '__main__':
       counter = TermFrequencyCounter()
       counter.add("加油加油中国队！")  # 第一个文档
       counter.add("中国观众高呼加油中国")  # 第二个文档
       for termFrequency in counter:  # 遍历每个词与词频
           print("%s=%d" % (termFrequency.getTerm(), termFrequency.getFrequency()))
       print(counter.top(2))  # 取 top N
   
       #  根据词频提取关键词
       print('')
       print(TermFrequencyCounter.getKeywordList("女排夺冠，观众欢呼女排女排女排！", 3))
   ```

   运行结果如下:

   ```
   中国=2
   中国队=1
   加油=3
   观众=1
   高呼=1
   [加油=3, 中国=2]
   
   [女排, 观众, 欢呼]
   ```

   用词频来提取关键词有一个缺陷，那就是高频词并不等价于关键词。比如在一个体育网站中，所有文章都是奥运会报道，导致“奥运会”词频最高，用户希望通过关键词看到每篇文章的特色。此时，TF-IDF 就派上用场了。

   

2. **TF-IDF**

   TF-IDF (Term Frequency-lnverse Document Frequency,词频-倒排文档频次)是信息检索中衡量一个词语重要程度的统计指标，被广泛用于Lucene、Solr、Elasticsearch 等搜索引擎。

   相较于词频，TF-IDF 还综合考虑词语的稀有程度。在TF-IDF计算方法中，一个词语的重要程度不光正比于它在文档中的频次，还反比于有多少文档包含它。包含该词语的文档越多，就说明它越宽泛， 越不能体现文档的特色。 正是因为需要考虑整个语料库或文档集合，所以TF-IDF在关键词提取时属于多文档方法。

<font color="red">
TF-IDF是一种统计方法，用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。

TF-IDF的主要思想是：如果某个单词在一篇文章中出现的频率TF高，并且在其他文章中很少出现，则认为此词或者短语具有很好的类别区分能力，适合用来分类。

TF为词频，IDF为逆文档频率。
    
计算公式如下:

$$TF-IDF(t,d)=\frac{TF(t,d)}{DF(t,d)}=TF(t,d) \times IDF(t,d)$$

$$TF(t,d)=\frac{词t在当前文档d中的词频}{当前文档d中词的总个数}$$

$$IDF(t,d)=log\frac{文档总数}{1+包含词t的文档数}$$

$$DF(t)=\frac{包含词t的文档数}{文档总数}$$

逆文档频率IDF分母中加入常数1是可选的，对于没有出现在任何训练样本中的词汇，它不能保证分母不为0；取对数是为了保证文档中出现频率较低的词汇不会被赋予过大的权重。
    
举例：“的”这个字在所有的文档中经常出现，那么我们通过计算词频-逆文档频率得出的值就比较小，这个值就代表有用或辨识度的大小。证明这个“的”字不具有辨识性。

其中，t代表单词，d代表当前文档，TF(t,d) 代表 t 在 d 中出现频次，DF(t) 代表有多少篇文档包含 t。DF 的倒数称为IDF，这也是 TF-IDF 得名的由来。

当然，实际应用时做一些扩展，比如加一平滑、IDF取对数以防止浮点数下溢出。
    
</font> 
   HanLP的示例如下:

   ```python
   from pyhanlp import *
   
   TfIdfCounter = JClass('com.hankcs.hanlp.mining.word.TfIdfCounter')
   
   if __name__ == '__main__':
       counter = TfIdfCounter()
       counter.add("《女排夺冠》", "女排北京奥运会夺冠")  # 输入多篇文档
       counter.add("《羽毛球男单》", "北京奥运会的羽毛球男单决赛")
       counter.add("《女排》", "中国队女排夺北京奥运会金牌重返巅峰，观众欢呼女排女排女排！")
       counter.compute()  # 输入完毕
       for id in counter.documents():
           print(id + " : " + counter.getKeywordsOf(id, 3).toString())  # 根据每篇文档的TF-IDF提取关键词
       # 根据语料库已有的IDF信息为语料库之外的新文档提取关键词
       
       print('')
       print(counter.getKeywords("奥运会反兴奋剂", 2))
   ```

   运行后如下:

   ```
   《女排》 : [女排=5.150728289807123, 重返=1.6931471805599454, 巅峰=1.6931471805599454]
   《女排夺冠》 : [夺冠=1.6931471805599454, 女排=1.2876820724517808, 奥运会=1.0]
   《羽毛球男单》 : [决赛=1.6931471805599454, 羽毛球=1.6931471805599454, 男单=1.6931471805599454]
   
   [反, 兴奋剂]
   ```

   观察输出结果，可以看出 TF-IDF 有效的避免了给予“奥运会”这个宽泛的词语过高的权重。

   TF-IDF在大型语料库上的统计类似于一种学习过程，假如我们没有这么大型的语料库或者存储IDF的内存，同时又想改善词频统计的效果该怎么办呢？此时可以使用TextRank算法。

   

3. **TextRank**

   TextRank 是 PageRank 在文本中的应用，PageRank是一种用于排序网页的随机算法，它的工作原理是将互联网看作有向图，互联网上的网页视作节点，节点 Vi 到节点 Vj 的超链接视作有向边，初始化时每个节点的权重 S(Vi) 都是1，以迭代的方式更新每个节点的权重。每次迭代权重的更新表达式如下:
   
   $$S\left(V_{i}\right)=(1-d)+d * \sum_{V_{j} \in \operatorname{In}\left(V_{i}\right)} \frac{1}{\left|\operatorname{Out}\left(V_{j}\right)\right|} S\left(V_{j}\right)$$
   
   其中 d 是一个介于 (0,1) 之间的常数因子，在PagRank中模拟用户点击链接从而跳出当前网站的概率，In(Vi) 表示链接到 Vi 的节点集合，Out(Vj) 表示从 Vj 出发链接到的节点集合。可见，开不是外链越多，网站的PageRank就越高。网站给别的网站做外链越多，每条外链的权重就越低。如果一个网站的外链都是这种权重很低的外链，那么PageRank也会下降，造成不良反应。正所谓物以类聚，垃圾网站推荐的链接往往也是垃圾网站。因此PageRank能够比较公正的反映网站的排名。

   将 PageRank 应用到关键词提取，无非是将单词视作节点而已，另外，每个单词的外链来自自身前后固定大小的窗口内的所有单词。

   HanLP实现的代码如下:

   ```python
   from pyhanlp import *
   
   """ 关键词提取"""
   content = (
   "程序员(英文Programmer)是从事程序开发、维护的专业人员。"
   "一般将程序员分为程序设计人员和程序编码人员，"
   "但两者的界限并不非常清楚，特别是在中国。"
   "软件从业人员分为初级程序员、高级程序员、系统"
   "分析员和项目经理四大类。")
   
   TextRankKeyword = JClass("com.hankcs.hanlp.summary.TextRankKeyword")
   keyword_list = HanLP.extractKeyword(content, 5)
   print(keyword_list)
   ```

   运行结果如下:

   ```
   [程序员, 程序, 分为, 人员, 软件]
   ```

<font color="red">

**pagerank（有向无环图）**

> 有向无环图（DAG）：在图论中，如果一个有向图从任意顶点出发无法经过若干条边回到该点，则这个图是一个有向无环图（DAG, Directed Acyclic Graph）。   
>
>无向图的边是没方向的，即两个相连的顶点可以互相抵达。而有向图的边是有方向的，即两个相连的顶点，根据边的方向，只能由一个顶点通向另一个顶点。（当然，如果两个相互指向对方的方向，那么它俩是互通的。）  
    
TextRank算法的思想来源于google的PageRank算法，其主要是将文本分割成若干的组成单元(单词、句子)并建立图模型，利用投票机制对文本中的重要成分进行排序，仅仅利用单文档的信息就可以完成关键词抽取和信息摘要的任务。和LDA、HMM等模型不同, TextRank不需要事先对多篇文档进行学习训练, 因其简洁有效而得到广泛应用。

PageRank,即网页排名，又称网页级别、Google左侧排名或佩奇排名。是Google创始人拉里·佩奇和谢尔盖·布林于1997年构建早期的搜索系统原型时提出的链接分析算法，自从Google在商业上获得空前的成功后，该算法也成为其他搜索引擎和学术界十分关注的计算模型。

PageRank的计算充分利用了两个假设：数量假设和质量假设。步骤如下：

在初始阶段：网页通过链接关系构建起Web图，每个页面设置相同的PageRank值，通过若干轮的计算，会得到每个页面所获得的最终PageRank值。随着每一轮的计算进行，网页当前的PageRank值会不断得到更新。

在一轮中更新页面PageRank得分的计算方法：在一轮更新页面PageRank得分的计算中，每个页面将其当前的PageRank值平均分配到本页面包含的出链上，这样每个链接即获得了相应的权值。而每个页面将所有指向本页面的入链所传入的权值求和，即可得到新的PageRank得分。当每个页面都获得了更新后的PageRank值，就完成了一轮PageRank计算。

对于一个页面A，则他的PR值为 "指向页面A的所有页面每个的权重值除以它的外链个数的累加和"：
    
$$S\left(V_{i}\right)=(1-d)+d * \sum_{V_{j} \in \operatorname{In}\left(V_{i}\right)} \frac{1}{\left|\operatorname{Out}\left(V_{j}\right)\right|} S\left(V_{j}\right)$$

在以上公式中：

$S(V_i)$: 网页$V_i$的重要度（权重），初始值可设为1。【在这里就是网页A的重要度】
    
$d$： 阻尼系数，一般为0.85。
    
$In(V_i)$：能跳转到网页$V_i$的页面，在图中对应入度对应的点。【在这里就是指向网页A的网页，也就是其他网页上的外链有网页A】
    
$|Out(V_j)|$：网页$V_j$能够跳转到的页面个数，在图中对应出度的点。【指向网页A的网页，它的外链个数】

$S(V_j)$：网页$V_j$的重要度（权重）【指向网页A的网页，它的重要度】

另外一个版本的公式:
    
$$S\left(V_{i}\right)=\frac{(1-d)}{N}+d * \sum_{V_{j} \in \operatorname{In}\left(V_{i}\right)} \frac{1}{\left|\operatorname{Out}\left(V_{j}\right)\right|} S\left(V_{j}\right), \text { 其中 } N \text { 为页面总数 }$$

**例子**
    
<img src="imgs/pagerank.png" width="300" />
    
为了便于计算，我们假设每个页面的PR初始值为1，d为0.5。接下来我们针对每个节点计算对应的PageRank值:

页面A的PR值计算:

$$PR(A)=0.5+0.5∗PR(C)=0.5+0.5=1$$
    
页面B的PR值计算:
    
$$PR(B)=0.5+0.5∗(PR(A)÷2)=0.5+0.5⋅0.5=0.75$$
    
页面C的PR值计算:
    
$$PR(C)=0.5+0.5∗(PR(A)÷2+PR(B))=0.5+0.5⋅(0.5+0.75)=1.125$$

从以上的计算可以看出，每个节点投票只能投一次，但是由于其指向其他节点多了即节点的出度(C(T))，其对每个节点的投票需要加权，加权值为$\frac{1}{C(T)}$。在上述的计算过程中，节点A对B节点的投票值只有1/2的权重，因为A节点的出度为2。其他的依次类推。
入度表示每个节点被多少节点指向，其PR值为这些入度的路径的PR值求和。

下面是迭代计算12轮之后，各个页面的PR值：
    
<img src="imgs/pagerank_result.png" width="400"/>

既然算法需要迭代训练，那么肯定需要知道什么时候收敛。所以一般需要设置收敛条件：比如上次迭代结果与本次迭代结果小于某个误差；比如还可以设置最大循环次数。

我们这样计算时比较麻烦，我们可以将这个图用邻接矩阵来表示。

将上面的图用邻接矩阵表示：
    
$$
W=\begin{matrix}
0 &0 &1 \\ 
1 &0 &0 \\
1 &1 &0 \\
\end{matrix}
$$

第一列代表A是否指向A、B、C，第一列第一行0代表A不指向A，第一列第二行1代表A指向B，第一列第三行1代表A指向C。
    
第二列代表B是否指向A、B、C，第二列第一行0代表B不指向A，第二列第二行0代表B不指向B，第二列第三行1代表B指向C。
    
第二列代表C是否指向A、B、C，第三列第一行1代表C指向A，第三列第二行0代表C不指向B，第三列第三行1代表C不指向C。

当用作计算时，这里的值就可以代表页面A指向页面B，带给B带来的权重占比，对列做归一化即可，其实就是求出$\frac{1}{\left|\operatorname{Out}\left(V_{j}\right)\right|}$：
    
$$
W=\begin{matrix}
0 &0 &1 \\ 
0.5 &0 &0 \\
0.5 &1 &0 \\
\end{matrix}
$$    
    
初始化权重值$S_0$（这里的权重值就是PR值）：

$$S_0=[1,1,1]$$
    
这3个分别是节点A、B、C的初始权重值（PR值）

所以：

$$S_1=1-d+d(W.S_0^T)$$

$$S_{i+1}=1-d+d(W.S_i^T)$$
</font>

In [34]:
import numpy as np

W = np.array([[0,0,1],[0.5,0,0],[0.5,1,0]])
S = np.array([1,1,1])
d = 0.5
W = W/W.sum(axis=0)
# print(np.matmul(W,S.T))
for i in range(20):
    S=1-d+d*(np.matmul(W,S.T))
    print(S)

[1.   0.75 1.25]
[1.125 0.75  1.125]
[1.0625  0.78125 1.15625]
[1.078125 0.765625 1.15625 ]
[1.078125   0.76953125 1.15234375]
[1.07617188 0.76953125 1.15429688]
[1.07714844 0.76904297 1.15380859]
[1.0769043  0.76928711 1.15380859]
[1.0769043  0.76922607 1.15386963]
[1.07693481 0.76922607 1.15383911]
[1.07691956 0.7692337  1.15384674]
[1.07692337 0.76922989 1.15384674]
[1.07692337 0.76923084 1.15384579]
[1.07692289 0.76923084 1.15384626]
[1.07692313 0.76923072 1.15384614]
[1.07692307 0.76923078 1.15384614]
[1.07692307 0.76923077 1.15384616]
[1.07692308 0.76923077 1.15384615]
[1.07692308 0.76923077 1.15384615]
[1.07692308 0.76923077 1.15384615]


<font color="red">

**TODO pagerank存在两个问题：Dead Ends（节点死亡：存在一些节点没有外链导致多次迭代后权重值S为0）和Spider Traps（爬虫陷阱：某些网页将自己设为外链），阻尼可以解决Dead Ends的问题**
https://zhuanlan.zhihu.com/p/197877312

    
**基于TextRank的关键词提取**
    
textrank和pagerank的原理是一样的，计算公式是一样的。区别在于：textrank是无向无环图，即共现词之间是相互指向的（相互连通的），对于在共现窗口里的词汇可以建立相互连通关系。
    
关键词抽取的任务就是从一段给定的文本中自动抽取出若干有意义的词语或词组。TextRank算法是利用局部词汇之间关系（共现窗口）对后续关键词进行排序，直接从文本本身抽取。其主要步骤如下：

1. 把给定的文本T按照完整句子进行分割，即
    
2. 对于每个句子，进行分词和词性标注处理，并过滤掉停用词，只保留指定词性的单词，如名词、动词、形容词，即，其中是保留后的候选关键词。
    
3. 构建候选关键词图G = (V,E)，其中V为节点集，由（2）生成的候选关键词组成，然后采用共现关系（co-occurrence）构造任两点之间的边，两个节点之间存在边仅当它们对应的词汇在长度为K的窗口中共现，K表示窗口大小，即最多共现K个单词。

解释：设置一个窗口k，${w_1,w_2,w_3......w_k}$对于在滑动窗口k之内的词，都可以认为是有联系的，也就是任意两个词都是联通的，对于有联系的两个词，可以在有词构成的图中增加相应的权重。一个窗口遍历完文章的所有句子后，一个由词语构成的图就建立了。
    
4. 根据上面公式，迭代传播各节点的权重，直至收敛。
    
5. 对节点权重进行倒序排序，从而得到最重要的T个单词，作为候选关键词。
    
6. 由（5）得到最重要的T个单词，在原始文本中进行标记，若形成相邻词组，则组合成多词关键词。例如，文本中有句子“Matlab code for plotting ambiguity function”，如果“Matlab”和“code”均属于候选关键词，则组合成“Matlab code”加入关键词序列。

```java
    /**
     * 使用已经分好的词来计算rank
     *
     * @param termList
     * @return
     */
    public Map<String, Float> getTermAndRank(List<Term> termList)
    {
        List<String> wordList = new ArrayList<String>(termList.size());
        for (Term t : termList)
        {
            if (shouldInclude(t))
            {
                wordList.add(t.word);
            }
        }
//        System.out.println(wordList);
        Map<String, Set<String>> words = new TreeMap<String, Set<String>>();
        Queue<String> que = new LinkedList<String>(); //该队列维护窗口为5的5个同时出现的词。
        for (String w : wordList)
        {
            if (!words.containsKey(w))
            {
                words.put(w, new TreeSet<String>());
            }
            // 复杂度O(n-1)
            if (que.size() >= 5)//窗口为5
            {
                que.poll();//remove，取出 LinkedList 的第一个元素，并将该元素删除
            }
            for (String qWord : que)
            {
                if (w.equals(qWord))
                {
                    continue;
                }
                //既然是邻居,那么关系是相互的,遍历一遍即可
                words.get(w).add(qWord);
                words.get(qWord).add(w);
            }
            que.offer(w);//LinkedList尾部追加
        }
//        System.out.println(words);
        Map<String, Float> score = new HashMap<String, Float>();
        //依据TF来设置初值
        for (Map.Entry<String, Set<String>> entry : words.entrySet())
        {
            score.put(entry.getKey(), sigMoid(entry.getValue().size()));
        }
        for (int i = 0; i < max_iter; ++i)
        {
            Map<String, Float> m = new HashMap<String, Float>();
            float max_diff = 0;
            for (Map.Entry<String, Set<String>> entry : words.entrySet())
            {
                String key = entry.getKey();
                Set<String> value = entry.getValue();
                m.put(key, 1 - d);
                for (String element : value)
                {
                    int size = words.get(element).size();
                    if (key.equals(element) || size == 0) continue;
                    m.put(key, m.get(key) + d / size * (score.get(element) == null ? 0 : score.get(element)));
                }
                max_diff = Math.max(max_diff, Math.abs(m.get(key) - (score.get(key) == null ? 0 : score.get(key))));
            }
            score = m;
            if (max_diff <= min_diff) break;
        }

        return score;
    }

    /**
     * sigmoid函数
     *
     * @param value
     * @return
     */
    public static float sigMoid(float value)
    {
        return (float) (1d / (1d + Math.exp(-value)));
    }    
```    

<img src="imgs/textrank_keyword_treemap.png" width="400"/>    
    
<img src="imgs/textrank_keyword_score.png" width="400"/>    

阻尼
邻接矩阵
</font>




### 9.3 短语提取

<font color="red">在信息抽取领域，另一项重要的任务就是提取中文短语，也即固定多字词表达串的识别。短语提取经常用于搜索引擎的自动推荐，文档的简介生成等。

利用互信息和左右信息熵，我们可以轻松地将新词提取算法拓展到短语提取。只需将新词提取时的字符替换为单词， 字符串替换为单词列表即可。为了得到单词，我们依然需要进行中文分词【先进行分词】。 大多数时候， 停用词对短语含义表达帮助不大，所以通常在分词后过滤掉。</font>

代码如下:

```python
from pyhanlp import *

""" 短语提取"""
text = '''
  算法工程师
  算法（Algorithm）是一系列解决问题的清晰指令，也就是说，能够对一定规范的输入，在有限时间内获得所要求的输出。
  如果一个算法有缺陷，或不适合于某个问题，执行这个算法将不会解决这个问题。不同的算法可能用不同的时间、
  空间或效率来完成同样的任务。一个算法的优劣可以用空间复杂度与时间复杂度来衡量。算法工程师就是利用算法处理事物的人。

  1职位简介
  算法工程师是一个非常高端的职位；
  专业要求：计算机、电子、通信、数学等相关专业；
  学历要求：本科及其以上的学历，大多数是硕士学历及其以上；
  语言要求：英语要求是熟练，基本上能阅读国外专业书刊；
  必须掌握计算机相关知识，熟练使用仿真工具MATLAB等，必须会一门编程语言。

  2研究方向
  视频算法工程师、图像处理算法工程师、音频算法工程师 通信基带算法工程师

  3目前国内外状况
  目前国内从事算法研究的工程师不少，但是高级算法工程师却很少，是一个非常紧缺的专业工程师。
  算法工程师根据研究领域来分主要有音频/视频算法处理、图像技术方面的二维信息算法处理和通信物理层、
  雷达信号处理、生物医学信号处理等领域的一维信息算法处理。
  在计算机音视频和图形图像技术等二维信息算法处理方面目前比较先进的视频处理算法：机器视觉成为此类算法研究的核心；
  另外还有2D转3D算法(2D-to-3D conversion)，去隔行算法(de-interlacing)，运动估计运动补偿算法
  (Motion estimation/Motion Compensation)，去噪算法(Noise Reduction)，缩放算法(scaling)，
  锐化处理算法(Sharpness)，超分辨率算法(Super Resolution) 手势识别(gesture recognition) 人脸识别(face recognition)。
  在通信物理层等一维信息领域目前常用的算法：无线领域的RRM、RTT，传送领域的调制解调、信道均衡、信号检测、网络优化、信号分解等。
  另外数据挖掘、互联网搜索算法也成为当今的热门方向。
  算法工程师逐渐往人工智能方向发展。'''
 
phrase_list = HanLP.extractPhrase(text, 5)
print(phrase_list)

```

运行结果如下:

```
[算法工程师, 算法处理, 一维信息, 算法研究, 信号处理]
```

目前该模块只支持提取二元语法短语。在另一些场合，关键词或关键短语依然显得碎片化，不足以表达完整的主题。这时通常提取中心句子作为文章的简短摘要，而关键句的提取依然是基于 PageRank 的拓展。



### TODO 9.4 关键句提取

由于一篇文章中几乎不可能出现相同的两个句子，所以朴素的 PageRank 在句子颗粒度上行不通。为了将 PageRank 利用到句子颗粒度上去，我们引人 BM25 算法衡量句子的相似度，改进链接的权重计算。这样窗口的中心句与相邻的句子间的链接变得有强有弱，相似的句子将得到更高的投票。而文章的中心句往往与其他解释说明的句子存在较高的相似性，这恰好为算法提供了落脚点。本节将先介绍BM25算法，后介绍TextRank在关键句提取中的应用。

1. **BM25**

   在信息检索领域中，BM25 是TF-IDF的一种改进变种。TF-IDF衡量的是单个词语在文档中的重要程度，而在搜索引擎中，查询串(query)往往是由多个词语构成的。如何衡量多个词语与文档的关联程度，就是BM25所解决的问题。

   形式化的定义 Q 为查询语句，由关键字 q1 到 qn 组成，D 为一个被检索的文档，BM25度量如下:
   
   $$\operatorname{BM} 25(D, Q)=\sum_{i=1}^{n} \operatorname{IDF}\left(q_{i}\right) \cdot \frac{\operatorname{TF}\left(q_{i}, D\right) \cdot\left(k_{1}+1\right)}{\operatorname{TF}\left(q_{i}, D\right)+k_{1} \cdot\left(1-b+b \cdot \frac{|D|}{\operatorname{avg} D L}\right)}$$
   
2. **TextRank**

   有了BM25算法之后，将一个句子视作查询语句，相邻的句子视作待查询的文档，就能得到它们之间的相似度。以此相似度作为 PageRank 中的链接的权重，于是得到一种改进算法，称为TextRank。它的形式化计算方法如下:
   
   $$\operatorname{WS}\left(V_{i}\right)=(1-d)+d \times \sum_{V_{j} \in \ln \left(V_{i}\right)} \frac{\operatorname{BM} 25\left(V_{i}, V_{j}\right)}{\sum_{V_{k} \in \operatorname{Out}\left(V_{j}\right)} \mathrm{Bu} 2 \mathrm{~s}\left(V_{k}, V_{j}\right)} \operatorname{WS}\left(V_{j}\right)$$
   
   其中，WS(Vi) 就是文档中第 i 个句子的得分，重复迭代该表达式若干次之后得到最终的分值，排序后输出前 N 个即得到关键句。代码如下:

   ```python
   from pyhanlp import *
   
   """自动摘要"""
   document = '''水利部水资源司司长陈明忠9月29日在国务院新闻办举行的新闻发布会上透露，
   根据刚刚完成了水资源管理制度的考核，有部分省接近了红线的指标，
   有部分省超过红线的指标。对一些超过红线的地方，陈明忠表示，对一些取用水项目进行区域的限批，
   严格地进行水资源论证和取水许可的批准。'''
   
   TextRankSentence = JClass("com.hankcs.hanlp.summary.TextRankSentence")
   sentence_list = HanLP.extractSummary(document, 3)
   print(sentence_list)
   ```

   结果如下:

   ```
   [严格地进行水资源论证和取水许可的批准, 水利部水资源司司长陈明忠9月29日在国务院新闻办举行的新闻发布会上透露, 有部分省超过红线的指标]
   ```



### 9.5 总结

<font color="red">我们看到，新词提取与短语提取，关键词与关键句的提取，在原理上都是同一种算法在不同文本颗粒度上的应用。值得一提的是， 这些算法都不需要标注语料的参与，满足了人们“不劳而获”的欲望。然而必须指出的是，这些算法的效果非常有限。**对于同一个任务，监督学习方法的效果通常远远领先于无监督学习方法。**</font>